In [19]:
import numpy as np
import pandas as pd
from scipy.linalg import fractional_matrix_power

In [20]:
def normalize_matrix(A: np.array) -> np.array:
    return (A - A.min(axis=1)[:,None]) / (A.max(axis=1)[:,None] - A.min(axis=1)[:,None])

In [21]:
def calculate_mean_and_covariance_matrices(X: np.array, Y: np.array) -> (np.array, np.array, np.array, np.array ,np.array):
    assert X.shape[1] == Y.shape[1]
    n = X.shape[1]
    mu_X, mu_Y = np.mean(X, axis=1), np.mean(Y, axis=1)
    X_c, Y_c = X - mu_X[:, None], Y - mu_Y[:, None]
    return mu_X, mu_Y, np.dot(X_c, X_c.T) / n, np.dot(Y_c, X_c.T) / n, np.dot(Y_c, Y_c.T) / n

In [22]:
def reduced_rank_regression(X: np.array, Y: np.array, Gamma: np.array, t: int) -> (np.array, np.array, np.array, float):
    assert t <= Y.shape[0]
    assert Y.shape[0] <= X.shape[0]
    assert X.shape[1] == Y.shape[1]
    assert Gamma.shape == (Y.shape[0], Y.shape[0])
    mu_X, mu_Y, Sigma_XX, Sigma_YX, Sigma_YY = calculate_mean_and_covariance_matrices(X, Y)
    U, S, _ = np.linalg.svd(np.dot(np.dot(fractional_matrix_power(Gamma, 0.5), Sigma_YX), 
                                   fractional_matrix_power(Sigma_XX, -0.5)))
    C_min = np.dot(np.dot(np.dot(fractional_matrix_power(Gamma, -0.5), np.dot(U[:,0:t], U[:,0:t].T)), 
                   fractional_matrix_power(Gamma, 0.5)), np.dot(Sigma_YX, np.linalg.inv(Sigma_XX)))
    mu_min = mu_Y - np.dot(C_min, mu_X)
    min_value = np.trace(Sigma_YY) * np.trace(Gamma) - np.sum(S[0:t])
    return mu_min, C_min, S, min_value

In [26]:
wine_dataset = pd.read_csv("wine_dataset.csv")

In [27]:
X_variables = ['residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'sulphates', 'alcohol']
Y_variables = ['volatile acidity', 'fixed acidity', 'citric acid']
X = normalize_matrix(wine_dataset[X_variables].values.T)
Y = normalize_matrix(wine_dataset[Y_variables].values.T)

In [28]:
t = 2
rrr_result_values = reduced_rank_regression(X, Y, np.identity(Y.shape[0]), t)
rrr_result_values

(array([ 0.39720737, -0.26818126, -0.33256443]),
 array([[-0.02181387, -0.04990488,  0.02835088, -0.08233195, -0.03252975,
         -0.22928008, -0.1359444 ],
        [-0.34801385,  0.12470403, -0.09366844,  0.06023219,  1.05333682,
         -0.04370523,  0.39824929],
        [-0.20312143,  0.16750855, -0.11083069,  0.17646172,  0.77652181,
          0.34625592,  0.49649868]]),
 array([0.15617356, 0.05126302, 0.0329217 ]),
 0.02260760491195482)

In [29]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import *
%matplotlib qt

In [30]:
fig = plt.figure()
fig.set_size_inches(16, 16)
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, 1)
ax.set_ylim3d(0, 1)
ax.set_zlim3d(0, 1)
ax.invert_xaxis()

# Plotte die Datenpunkte
xpts, ypts, zpts = Y[0,:], Y[1,:], Y[2,:]
ax.scatter(xpts, ypts, zpts, c=zpts)
ax.set_xlabel('x'), ax.set_ylabel('y'), ax.set_zlabel('z')

mu_min, C_min, _, _ = rrr_result_values
predY = mu_min[:,None] + np.dot(C_min, X)
xpts, ypts, zpts = predY[0,:], predY[1,:], predY[2,:]
ax.scatter(xpts, ypts, zpts, c='black')

if t == 2:
    # Plotte mögliche Werte für die Approximation mit t=2
    point, C_min, _, _ = rrr_result_values
    normal = np.cross(C_min[:,0], C_min[:,1])
    d = -point.dot(normal)
    xx, yy = np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))
    z = (-normal[0] * xx - normal[1] * yy - d) / normal[2]
    ax.plot_surface(xx, yy, z, alpha=0.25)
fig.tight_layout()

In [23]:
X = np.random.normal(0, 0.3, size=(3, 1000))
C = 0.25 * np.array([[1, 0, 1],
                     [0, 1, 0],
                     [0, 0, -1]])
mu = np.array([0.5, 0.5, 0.5])
Y = mu[:,None] + np.dot(C, X) + np.random.normal(0, 0.15, size=(3, 1000))

In [24]:
t = 3
rrr_result_values = reduced_rank_regression(X, Y, np.identity(Y.shape[0]), t)
rrr_result_values

(array([0.50488828, 0.49014634, 0.49952944]),
 array([[ 0.24645582, -0.01865384,  0.24105942],
        [-0.01830476,  0.21251471,  0.00748936],
        [-0.01795381, -0.017466  , -0.27940704]]),
 array([0.12361606, 0.06524274, 0.04394684]),
 0.030500676791800796)

In [25]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import *
%matplotlib qt

In [26]:
fig = plt.figure()
fig.set_size_inches(8, 8)
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, 1)
ax.set_ylim3d(0, 1)
ax.set_zlim3d(0, 1)
ax.invert_xaxis()

# Plotte die Datenpunkte
xpts, ypts, zpts = Y[0,:], Y[1,:], Y[2,:]
ax.scatter(xpts, ypts, zpts, c=zpts)
ax.set_xlabel('x'), ax.set_ylabel('y'), ax.set_zlabel('z')

mu_min, C_min, _, _ = rrr_result_values
predY = mu_min[:,None] + np.dot(C_min, X)
xpts, ypts, zpts = predY[0,:], predY[1,:], predY[2,:]
ax.scatter(xpts, ypts, zpts, c='black')

if t == 2:
    # Plotte mögliche Werte für die Approximation mit t=2
    point, C_min, _, _ = rrr_result_values
    normal = np.cross(C_min[:,0], C_min[:,1])
    d = -point.dot(normal)
    xx, yy = np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))
    z = (-normal[0] * xx - normal[1] * yy - d) / normal[2]
    ax.plot_surface(xx, yy, z, alpha=0.2, color='grey')
fig.tight_layout()

plt.savefig('X_Y_{}_simulated.png'.format(t), dpi=200)